In [2]:
import numpy as np
import pandas as pd
import os
#import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from scipy import sparse as sps
from sklearn.metrics import mean_squared_error
from ast import literal_eval
from scipy.sparse.linalg import svds
import statistics
from math import sqrt
from scipy import sparse

In [ ]:
data_path = 'datasets/'
ratings_filename = 'ratings.csv'

ratings_df = pd.read_csv(os.path.join(data_path, ratings_filename), usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
ratings_df.head()

In [ ]:
ratings_pivot_df = ratings_df.pivot(index='userId',columns='movieId',values='rating')
ratings_pivot_df.head()

In [3]:
pivot_ratings = ratings_pivot_df.fillna(0)
pivot_ratings.head()

NameError: name 'ratings_pivot_df' is not defined

In [ ]:
pivot_ratings.shape # poklapa se sa brojem jedinstvenih korisnika odnosno filmova

In [ ]:
ratings_csr_matrix = sps.csr_matrix(pivot_ratings)
print(ratings_csr_matrix.todense()) #toarray returns an ndarray; todense returns a matrix.

In [ ]:
ratings_csr_matrix.shape

In [ ]:
plt.spy(ratings_csr_matrix.toarray())
#plt.axis('equal')
plt.xlim(0, 9724)
plt.ylim(0, 3000)
plt.show()

In [4]:
def CosineSimilarity(docVector1, docVector2):
    dotProduct = 0.0
    magnitude1 = 0.0
    magnitude2 = 0.0
    cosineSimilarity = 0.0

    for i in range(0, len(docVector1) - 1):
        dotProduct += docVector1[i] * docVector2[i]
        magnitude1 += np.power(docVector1[i], 2)
        magnitude2 += np.power(docVector2[i], 2)
    
    magnitude1 = np.sqrt(magnitude1)
    magnitude2 = np.sqrt(magnitude2)

    if(magnitude1 != 0.0 and magnitude2 != 0.0):
        cosineSimilarity = dotProduct / (magnitude1 * magnitude2)

    return(str(cosineSimilarity))

In [ ]:
def JaccardSimilarity(docVector1, docVector2):
    dotProduct = 0.0
    magnitude1 = 0.0
    magnitude2 = 0.0
    jaccardSimilarity = 0.0

    for i in range(0, len(docVector1) - 1):
        dotProduct += docVector1[i] * docVector2[i]
        magnitude1 += np.power(docVector1[i], 2)
        magnitude2 += np.power(docVector2[i], 2)

    if(magnitude1 != 0.0 and magnitude2 != 0.0):
        jaccardSimilarity = dotProduct / (magnitude1 + magnitude2 - dotProduct)

    return(str(jaccardSimilarity))

In [5]:
# Korigovana f-ja
def user_based_ratings_prediction_prim(u, i, users_similarity, ratings, k = 5):
    
    neighbors = []
    similarities = list(zip(users_similarity[u][:], range(users_similarity.shape[0])))  # broj korisnika

    similarities_sorted = sort_descending(similarities)

    # indeksi k najslicnijih
    for i in range(1, k + 1):
        neighbors.append(similarities_sorted[i][1])
    print ("neighbors: ", neighbors)

    rated_by_u = ratings[u].nonzero()[0]
    print("Ocenio korisnik u: ", rated_by_u)
    
    user_u_mean = 0
    user_u_mean = np.sum(ratings[u])
        
    if len(rated_by_u) != 0:
        user_u_mean = user_u_mean / len(rated_by_u) # prosecna ocena u-tog korisnika
        
    print("user_u_mean ", user_u_mean)
    
    
    numerator, denominator = 0.0, 0.0

    for v in neighbors:
        rated_by_v = ratings[v].nonzero()[0]
        user_v_mean = 0
        user_v_mean = np.sum(ratings[v])
        print("v ", v)
        print("user_v_mean ", user_v_mean)
        
        if len(rated_by_v) != 0:
            #for i in rated_by_v:
             #   user_v_mean += ratings[v, i]
            
            user_v_mean = user_v_mean / len(rated_by_v)
        
        r_vi = ratings[v,i]
        numerator += float((users_similarity[u][v]))*float((r_vi - user_v_mean))
        denominator += float(users_similarity[u][v])
    
    print("konacno ", user_u_mean + numerator/denominator)  
        
    return user_u_mean + numerator/denominator

In [6]:
def user_based_ratings_prediction_prim(u, i, users_similarity, ratings, k = 5):
    
    neighbors = []
    similarities = list(zip(users_similarity[u][:], range(users_similarity.shape[0])))  # broj korisnika

    similarities_sorted = sort_descending(similarities)

    # indeksi k najslicnijih
    for i in range(1, k + 1):
        neighbors.append(similarities_sorted[i][1])
    print ("neighbors: ", neighbors)

    rated_by_u = ratings[u].nonzero()[0]
    print("Ocenio korisnik u: ", rated_by_u)
    
    user_u_mean = 0
    user_u_mean = np.sum(ratings[u])
        
    if len(rated_by_u) != 0:
        user_u_mean = user_u_mean / len(rated_by_u) # prosecna ocena u-tog korisnika
        
    print("user_u_mean ", user_u_mean)
    
    
    numerator, denominator = 0.0, 0.0

    for v in neighbors:
        rated_by_v = ratings[v].nonzero()[0]
        user_v_mean = 0
        user_v_mean = np.sum(ratings[v])
        print("v ", v)
        print("user_v_mean ", user_v_mean)
        
        if len(rated_by_v) != 0:
            #for i in rated_by_v:
             #   user_v_mean += ratings[v, i]
            
            user_v_mean = user_v_mean / len(rated_by_v)
        
        r_vi = ratings[v,i]
        numerator += float((users_similarity[u][v]))*float((r_vi - user_v_mean))
        denominator += float(users_similarity[u][v])
    
    print("konacno ", user_u_mean + numerator/denominator)  
        
    return user_u_mean + numerator/denominator

In [ ]:
xy = ratings_csr_matrix
xy = xy.toarray()
xy

In [ ]:
# Jaccard
xy_filtered_matrix_2 = []
for y in xy:
    sum = 0
    nz = np.count_nonzero(y)
    if (nz > 1000):
        xy_filtered_matrix_2.append(y)
arr_shape_2 = np.vstack(xy_filtered_matrix_2).shape
filtered_arr_2 = np.vstack(xy_filtered_matrix_2)

cos_matrix_2 = []
for i in filtered_arr_2:
    a = []
    for j in filtered_arr_2:
        cos_sim = JaccardSimilarity(i, j)
        #print(cos_sim)
        a.append(float(cos_sim))
    cos_matrix_2.append(a)

arr2 = np.array(cos_matrix_2)
arr2.reshape( (12,12) )
type(arr2)

In [ ]:
user_based_ratings_prediction_prim(2, 1, arr2, filtered_arr_2, 5)

In [ ]:
# Cosine
xy_filtered_matrix = []
for y in xy:
    sum = 0
    nz = np.count_nonzero(y)
    if (nz > 1000):
        xy_filtered_matrix.append(y)
arr_shape = np.vstack(xy_filtered_matrix).shape
filtered_arr = np.vstack(xy_filtered_matrix)

cos_matrix = []
for i in filtered_arr:
    a = []
    for j in filtered_arr:
        cos_sim = CosineSimilarity(i, j)
        #print(cos_sim)
        a.append(float(cos_sim))
    cos_matrix.append(a)

arr = np.array(cos_matrix)
arr.reshape( (12,12) )
type(arr)

In [ ]:
user_based_ratings_prediction_prim(2, 1, arr, filtered_arr, 5)

In [ ]:
def get_prediction_prim (ratings, users_similarity, cf_type, k = 5):
    predictions = np.zeros(ratings.shape)
    
    if cf_type == 'user':
        
        for u in range(ratings.shape[0]):
            for i in range(ratings.shape[1]):
                predictions[u,i] = user_based_ratings_prediction_prim(u, i, users_similarity, ratings, k = 5)
                
    elif cf_type == 'item':
        
        for u in range(ratings.shape[0]):
            for i in range(ratings.shape[1]):
                predictions[u,i] = item_based_ratings_prediction(u, i, users_similarity, ratings, k)
                
    else:
        print("Greska! Tip mora biti user ili item.")
        return

    return predictions

In [ ]:
predictions = get_prediction_prim(filtered_arr, arr, 'user', 5)

In [ ]:
predictions

In [ ]:
xy = train_and_validation
xy = xy.toarray()
xy_filtered_matrix = []
for y in xy:
    sum = 0
    nz = np.count_nonzero(y)
    if (nz > 1000):
        xy_filtered_matrix.append(y)

#train_and_validation.toarray()
arr_shape = np.vstack(xy_filtered_matrix).shape
arr_shape
filtered_arr = np.vstack(xy_filtered_matrix)


#cos_matrix_train_and_validation = []
#for i in filtered_train_and_validation_arr:
#    a = []
#    for j in filtered_arr:
#        cos_sim = CosineSimilarity(i, j)
#        a.append(cos_sim)
#    cos_matrix_train_and_validation.append(a)
    
#arr = np.array(cos_matrix_train_and_validation)
#arr.reshape( (12,12) )
#type(arr)

In [ ]:
xy = train_and_validation
xy = xy.toarray()
xy_filtered_matrix = []
for y in xy:
    sum = 0
    nz = np.count_nonzero(y)
    if (nz > 1000):
        xy_filtered_matrix.append(y)

#train_and_validation.toarray()
arr_shape = np.vstack(xy_filtered_matrix).shape
arr_shape
filtered_arr = np.vstack(xy_filtered_matrix)


#cos_matrix_train_and_validation = []
#for i in filtered_train_and_validation_arr:
#    a = []
#    for j in filtered_arr:
#        cos_sim = CosineSimilarity(i, j)
#        a.append(cos_sim)
#    cos_matrix_train_and_validation.append(a)
    
#arr = np.array(cos_matrix_train_and_validation)
#arr.reshape( (12,12) )
#type(arr)

In [7]:
errors_user, predictions_user = selection(train_and_validation, k_values, 'user')

NameError: name 'selection' is not defined